In [1]:
# !pip install selenium

In [2]:
# !pip install webdriver-manager

In [3]:
# !pip install openpyxl

In [3]:
import selenium
selenium.__version__

'4.11.2'

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

options = webdriver.ChromeOptions()
# 창 띄우지 않기
# options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

# 소장품 검색 페이지
urls = f'https://www.museum.go.kr/site/main/relic/search/collectionList#######'
# driver(chrome) open
driver.get(urls)

time.sleep(2)
#card1 > li:nth-child(12) > a

elements = driver.find_elements(By.CLASS_NAME, 'thum_box')
for element in elements :
    print(element.get_attribute('href'))
    link = element.get_attribute('href')
    driver.execute_script('window.open('');')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    time.sleep(2)

    try :
        collection_tit = driver.find_element(By.CLASS_NAME, 'outveiw-tit').text
    except :
        collection_tit = '작품명 알 수 없음'
    print(collection_tit)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    break

# # last page number 얻어오기
# page_body = driver.find_element(By.XPATH, '//*[@id="contents-area"]/div[3]/div/div[3]/div/div[2]/a[2]')
# href_text = page_body.get_attribute('href')
# href_num = href_text.split("'")[1]
# print(href_text)
# print(href_num) # last page


# for page_url in href_num  :
# for text in page_body :
    # href_text = text.find_element(By.XPATH, '//*[@id="contents-area"]/div[3]/div/div[3]/div/div[2]/a[2]').get_attribute('href')
    # print(href_text)

https://www.museum.go.kr/site/main/relic/search/view?relicId=329
흑갈유 병
